<a href="https://colab.research.google.com/github/Apple03244/DACON/blob/main/%EA%B3%A0%EA%B0%9D_%EB%8C%80%EC%B6%9C%EB%93%B1%EA%B8%89_%EB%B6%84%EB%A5%98_%ED%95%B4%EC%BB%A4%ED%86%A4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 고객 대출등급 분류 해커톤

In [ ]:
import pandas as pd

In [ ]:
train=pd.read_csv('/content/train.csv')

In [ ]:
train.head(3)

,ID,대출금액,대출기간,근로기간,주택소유상태,연간소득,부채_대비_소득_비율,총계좌수,대출목적,최근_2년간_연체_횟수,총상환원금,총상환이자,총연체금액,연체계좌수,대출등급
0,TRAIN_00000,12480000,36 months,6 years,RENT,72000000,18.90,15,부채 통합,0,0,0.0,0.0,0.0,C
1,TRAIN_00001,14400000,60 months,10+ years,MORTGAGE,130800000,22.33,21,주택 개선,0,373572,234060.0,0.0,0.0,B
2,TRAIN_00002,12000000,36 months,5 years,MORTGAGE,96000000,8.60,14,부채 통합,0,928644,151944.0,0.0,0.0,A


In [ ]:
import sklearn.preprocessing as skpre
import sklearn.model_selection as skmod
import sklearn.pipeline as skpip
import sklearn.base as skbase
import sklearn.compose as skcom

In [ ]:
import matplotlib.pyplot as plt
import seaborn as sb

In [ ]:
train.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 96294 entries, 0 to 96293
Data columns (total 15 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            96294 non-null  object 
 1   대출금액          96294 non-null  int64  
 2   대출기간          96294 non-null  object 
 3   근로기간          96294 non-null  object 
 4   주택소유상태        96294 non-null  object 
 5   연간소득          96294 non-null  int64  
 6   부채_대비_소득_비율   96294 non-null  float64
 7   총계좌수          96294 non-null  int64  
 8   대출목적          96294 non-null  object 
 9   최근_2년간_연체_횟수  96294 non-null  int64  
 10  총상환원금         96294 non-null  int64  
 11  총상환이자         96294 non-null  float64
 12  총연체금액         96294 non-null  float64
 13  연체계좌수         96294 non-null  float64
 14  대출등급          96294 non-null  object 
dtypes: float64(4), int64(5), object(6)
memory usage: 11.0+ MB


In [ ]:
train["대출기간"].value_counts()

 36 months    64479
 60 months    31815
Name: 대출기간, dtype: int64

In [ ]:
train["근로기간"].value_counts()

In [ ]:
train["대출목적"].value_counts()

In [ ]:
train["주택소유상태"].value_counts()

In [ ]:
train["최근_2년간_연체_횟수"].value_counts()

### 1. 전처리

In [ ]:
import numpy as np

#근로기간 전처리
class makecate_1(skbase.TransformerMixin,skbase.BaseEstimator):
  def __init__(self) -> None:
    super().__init__()
  def fit(self,X,y=None):
    self.columns_names=list(X.columns)
    return self
  def get_feature_names_out(self):
    return self.columns_names

  def transform(self,X,y=None):
    def trans(data):
      if data=="Unknown":
        return np.nan

      # 여기가 까다로운 부분
      condi_1="<" in data
      condi_2="1" in data
      condi_3="year" in data
      if all([condi_1,condi_2,condi_3]):
        return 0

      condi_4="+" in data
      condi_5="10" in data
      if all([condi_4,condi_5]):
        return 10
      else:
        result=[]
        for letter in data:
          if letter.isnumeric():
            result.append(letter)
        return int("".join(result))
    return X.apply(trans)


In [ ]:
train["대출목적"].unique()

array(['부채 통합', '주택 개선', '주요 구매', '휴가', '의료', '자동차', '신용 카드', '소규모 사업',
       '기타', '이사', '주택', '재생 에너지'], dtype=object)

In [ ]:
test.get_feature_names_out()

['근로기간']